In [1]:
import configparser
import json
import spotipy
import spotipy.util as util
import pandas as pd
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import bamboolib

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('../data/raw/spotify_creds.json') as f:
    spotify_creds = json.load(f)

In [3]:
client_id = spotify_creds['client_id']
client_secret = spotify_creds['client_secret']
#username = spotify_creds['username']
username = 'i9o5yf5bh59gq1nttz2iic006'
scope = spotify_creds['saved_library_scope']
redirect_uri = spotify_creds['saved_library_redirect_url']

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

In [5]:
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [7]:
sp.current_user()

{'display_name': 'Nihw',
 'external_urls': {'spotify': 'https://open.spotify.com/user/6oup6y9z3si0zmr49hq1kze94'},
 'followers': {'href': None, 'total': 1},
 'href': 'https://api.spotify.com/v1/users/6oup6y9z3si0zmr49hq1kze94',
 'id': '6oup6y9z3si0zmr49hq1kze94',
 'images': [{'height': None,
   'url': 'https://i.scdn.co/image/ab6775700000ee85f9b35ae5a6c28f047e6cc6c0',
   'width': None}],
 'type': 'user',
 'uri': 'spotify:user:6oup6y9z3si0zmr49hq1kze94'}

In [6]:
def connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri):
    
    client_credentials_manager = SpotifyClientCredentials(client_id, client_secret) 
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
        
    return sp

In [7]:
sp = connect_to_spotify_api(client_id, client_secret, username, scope, redirect_uri)

In [8]:
def get_saved_library():
    
    df_saved_tracks = pd.DataFrame()
    track_list = ''
    added_ts_list = []
    artist_list = []
    title_list = []
    popularity_list = []
    album_cover_list = []
    more_songs = True
    offset_index = 0

    while more_songs:
        songs = sp.current_user_saved_tracks(offset=offset_index)
        for song in songs['items']:
            #join track ids to a string for audio_features function
            track_list += song['track']['id'] +','
            #get the time when the song was added
            added_ts_list.append(song['added_at'])
            #get the title of the song
            title_list.append(song['track']['name'])
            #get popularity
            popularity_list.append(song['track']['popularity'])
            # get album cover
            album_cover_url = song['track']['album']['images'][0]['url']
            #response = requests.get(album_cover_url)
            #img = Image.open(BytesIO(response.content))
            album_cover_list.append(album_cover_url)
            #get all the artists in the song
            artists = song['track']['artists']
            artists_name = ''
            for artist in artists:
                artists_name += artist['name']  + ','
            artist_list.append(artists_name[:-1])
        track_list = ''
        if songs['next'] == None:
            # no more songs in playlist
            more_songs = False
        else:
            # get the next n songs
            offset_index += songs['limit']
    #include timestamp added, title and artists of a song
    df_saved_tracks['song_title'] = title_list
    df_saved_tracks['artists'] = artist_list
    df_saved_tracks['date_added'] = added_ts_list
    df_saved_tracks['album_cover'] = album_cover_list
    df_saved_tracks['popularity'] = popularity_list
    
    return df_saved_tracks

In [9]:
df = get_saved_library()

In [10]:
df

,song_title,artists,date_added,album_cover,popularity
0,Do It To It - Tiësto Remix,"ACRAZE,Tiësto,Cherish",2022-02-27T22:43:02Z,https://i.scdn.co/image/ab67616d0000b273db1c0e...,60
1,Freaky Deaky,"Tyga,Doja Cat",2022-02-27T20:40:22Z,https://i.scdn.co/image/ab67616d0000b2739d2729...,69
2,Nightmare,K-391,2022-02-26T19:55:18Z,https://i.scdn.co/image/ab67616d0000b2739f4a62...,45
3,If I Ain't Got You - Acoustic,John Adams,2022-02-21T05:09:51Z,https://i.scdn.co/image/ab67616d0000b273d2f483...,54
4,If I Ain't Got You,Alicia Keys,2022-02-21T05:09:25Z,https://i.scdn.co/image/ab67616d0000b27356ff19...,83
...,...,...,...,...,...
1100,Lullaby,"R3HAB,Mike Williams",2018-04-06T15:56:41Z,https://i.scdn.co/image/ab67616d0000b27342ccfe...,64
1101,Hold Me - AmPm Remix,"R3HAB,AmPm",2018-04-06T15:56:26Z,https://i.scdn.co/image/ab67616d0000b2732b245d...,20
1102,Low - R3HAB Remix,"Greyson Chance,R3HAB",2018-04-06T15:56:21Z,https://i.scdn.co/image/ab67616d0000b273f0073a...,39
1103,Ain't That Why - Skytech Remix,"R3HAB,Krewella,Skytech",2018-04-06T15:56:19Z,https://i.scdn.co/image/ab67616d0000b2734c49a1...,49
